# Amazon Sales Report Exploratory Data Analysis
<hr>

<!-- #### What is the distribution of the variables?
#### Are there any missing or inconsistent data points?
#### Are there outliers in the data?
#### What relationships exist between variables?
#### Are there any underlying patterns or trends in the data?
#### Addtional Findings -->

#### EDA Prep
<i>You do not need to exhaustedly explore|transform the data set in this step. Instead, you need to make data in numbers and scale them.</i>
<br><br>
From what I understand, is that we are keeping the dataset similar, but dropping what we might not need and making items numeric.

1. Load Dataset
2.

In [2]:
# 0: Imports
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [3]:
# 0.1: Helper Functions
def label_encoder_to_dict(label_encoder: LabelEncoder) -> dict:
    return dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

In [5]:
# 1: Loaded Dataset
df_amazon = pd.read_csv("dataset/original_amazon_sales_report.csv")
#print(df_amazon.columns)
df_amazon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   index               128975 non-null  int64  
 1   Order ID            128975 non-null  object 
 2   Date                128975 non-null  object 
 3   Status              128975 non-null  object 
 4   Fulfilment          128975 non-null  object 
 5   Sales Channel       128975 non-null  object 
 6   ship-service-level  128975 non-null  object 
 7   Style               128975 non-null  object 
 8   SKU                 128975 non-null  object 
 9   Category            128975 non-null  object 
 10  Size                128975 non-null  object 
 11  ASIN                128975 non-null  object 
 12  Courier Status      122103 non-null  object 
 13  Qty                 128975 non-null  int64  
 14  currency            121180 non-null  object 
 15  Amount              121180 non-nul

/var/folders/0s/r83z34657f316tyl6tlhslm80000gn/T/ipykernel_17038/1811533203.py:2: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df_amazon = pd.read_csv("dataset/original_amazon_sales_report.csv")


In [4]:
# 2. Drop the columns we don't think we'll use
df_amazon.drop(columns=[
    "index",
    "Order ID",
    "Sales Channel ",
    "Style",
    "SKU",
    "ASIN",
    "ship-city",
    "ship-state",
    "ship-postal-code",
    "ship-country",
    "fulfilled-by",
    "Unnamed: 22"
], inplace=True)

print(df_amazon.columns)
print(len(df_amazon.columns))

Index(['Date', 'Status', 'Fulfilment', 'ship-service-level', 'Category',
       'Size', 'Courier Status', 'Qty', 'currency', 'Amount', 'promotion-ids',
       'B2B'],
      dtype='object')
12


In [5]:
df_amazon.head()

,Date,Status,Fulfilment,ship-service-level,Category,Size,Courier Status,Qty,currency,Amount,promotion-ids,B2B
0,04-30-22,Cancelled,Merchant,Standard,Set,S,NaN,0,INR,647.62,NaN,False
1,04-30-22,Shipped - Delivered to Buyer,Merchant,Standard,kurta,3XL,Shipped,1,INR,406.00,Amazon PLCC Free-Financing Universal Merchant ...,False
2,04-30-22,Shipped,Amazon,Expedited,kurta,XL,Shipped,1,INR,329.00,IN Core Free Shipping 2015/04/08 23-48-5-108,True
3,04-30-22,Cancelled,Merchant,Standard,Western Dress,L,NaN,0,INR,753.33,NaN,False
4,04-30-22,Shipped,Amazon,Expedited,Top,3XL,Shipped,1,INR,574.00,NaN,False


In [6]:
# 3. Status - Get the unique entries and key them with a number
value_counts = df_amazon['Status'].value_counts()
print(value_counts)

# These are the status' we want to keep
entries_to_keep = ["Shipped", "Shipped - Delivered to Buyer", "Cancelled"]

# Filter the DataFrame to keep only the rows where the 'Status' column has one of the desired entries
df_amazon = df_amazon[df_amazon['Status'].isin(entries_to_keep)]
df_amazon = df_amazon.reset_index(drop=True)

# One hot Encoding
df_amazon = pd.get_dummies(df_amazon, columns=["Status"])
df_amazon.head()


Status
Shipped                          77804
Shipped - Delivered to Buyer     28769
Cancelled                        18332
Shipped - Returned to Seller      1953
Shipped - Picked Up                973
Pending                            658
Pending - Waiting for Pick Up      281
Shipped - Returning to Seller      145
Shipped - Out for Delivery          35
Shipped - Rejected by Buyer         11
Shipping                             8
Shipped - Lost in Transit            5
Shipped - Damaged                    1
Name: count, dtype: int64


,Date,Fulfilment,ship-service-level,Category,Size,Courier Status,Qty,currency,Amount,promotion-ids,B2B,Status_Cancelled,Status_Shipped,Status_Shipped - Delivered to Buyer
0,04-30-22,Merchant,Standard,Set,S,NaN,0,INR,647.62,NaN,False,True,False,False
1,04-30-22,Merchant,Standard,kurta,3XL,Shipped,1,INR,406.00,Amazon PLCC Free-Financing Universal Merchant ...,False,False,False,True
2,04-30-22,Amazon,Expedited,kurta,XL,Shipped,1,INR,329.00,IN Core Free Shipping 2015/04/08 23-48-5-108,True,False,True,False
3,04-30-22,Merchant,Standard,Western Dress,L,NaN,0,INR,753.33,NaN,False,True,False,False
4,04-30-22,Amazon,Expedited,Top,3XL,Shipped,1,INR,574.00,NaN,False,False,True,False


In [7]:
# 4. Fulfillment - Label Encode

# Unique Values
unique_fulfillment = df_amazon["Fulfilment"].unique()
print(unique_fulfillment)

# Counts of values
fulfillment_counts = df_amazon['Fulfilment'].value_counts()
print(fulfillment_counts)

df_amazon = pd.get_dummies(df_amazon, columns=["Fulfilment"])
df_amazon.head()



['Merchant' 'Amazon']
Fulfilment
Amazon      89275
Merchant    35630
Name: count, dtype: int64


,Date,ship-service-level,Category,Size,Courier Status,Qty,currency,Amount,promotion-ids,B2B,Status_Cancelled,Status_Shipped,Status_Shipped - Delivered to Buyer,Fulfilment_Amazon,Fulfilment_Merchant
0,04-30-22,Standard,Set,S,NaN,0,INR,647.62,NaN,False,True,False,False,False,True
1,04-30-22,Standard,kurta,3XL,Shipped,1,INR,406.00,Amazon PLCC Free-Financing Universal Merchant ...,False,False,False,True,False,True
2,04-30-22,Expedited,kurta,XL,Shipped,1,INR,329.00,IN Core Free Shipping 2015/04/08 23-48-5-108,True,False,True,False,True,False
3,04-30-22,Standard,Western Dress,L,NaN,0,INR,753.33,NaN,False,True,False,False,False,True
4,04-30-22,Expedited,Top,3XL,Shipped,1,INR,574.00,NaN,False,False,True,False,True,False


In [8]:
# 5. Ship-Service-Level: Label Encode

# Unique Values
unique_service = df_amazon["ship-service-level"].unique()
print(unique_service)

# Counts of values
service_counts = df_amazon["ship-service-level"].value_counts()
print(service_counts)

df_amazon = pd.get_dummies(df_amazon, columns=["ship-service-level"])
df_amazon.head()

['Standard' 'Expedited']
ship-service-level
Expedited    88202
Standard     36703
Name: count, dtype: int64


,Date,Category,Size,Courier Status,Qty,currency,Amount,promotion-ids,B2B,Status_Cancelled,Status_Shipped,Status_Shipped - Delivered to Buyer,Fulfilment_Amazon,Fulfilment_Merchant,ship-service-level_Expedited,ship-service-level_Standard
0,04-30-22,Set,S,NaN,0,INR,647.62,NaN,False,True,False,False,False,True,False,True
1,04-30-22,kurta,3XL,Shipped,1,INR,406.00,Amazon PLCC Free-Financing Universal Merchant ...,False,False,False,True,False,True,False,True
2,04-30-22,kurta,XL,Shipped,1,INR,329.00,IN Core Free Shipping 2015/04/08 23-48-5-108,True,False,True,False,True,False,True,False
3,04-30-22,Western Dress,L,NaN,0,INR,753.33,NaN,False,True,False,False,False,True,False,True
4,04-30-22,Top,3XL,Shipped,1,INR,574.00,NaN,False,False,True,False,True,False,True,False


In [9]:
# 6. Category - Label Encoding

# Unique Values
unique_categories = df_amazon["Category"].unique()
print(unique_categories)

# Counts of values
service_categories = df_amazon["Category"].value_counts()
print(service_categories)

# One hot encoding
df_amazon = pd.get_dummies(df_amazon, columns=["Category"])
df_amazon.head()

['Set' 'kurta' 'Western Dress' 'Top' 'Ethnic Dress' 'Bottom' 'Saree'
 'Blouse' 'Dupatta']
Category
Set              48646
kurta            48500
Western Dress    14792
Top              10339
Ethnic Dress      1134
Blouse             908
Bottom             421
Saree              162
Dupatta              3
Name: count, dtype: int64


,Date,Size,Courier Status,Qty,currency,Amount,promotion-ids,B2B,Status_Cancelled,Status_Shipped,...,ship-service-level_Standard,Category_Blouse,Category_Bottom,Category_Dupatta,Category_Ethnic Dress,Category_Saree,Category_Set,Category_Top,Category_Western Dress,Category_kurta
0,04-30-22,S,NaN,0,INR,647.62,NaN,False,True,False,...,True,False,False,False,False,False,True,False,False,False
1,04-30-22,3XL,Shipped,1,INR,406.00,Amazon PLCC Free-Financing Universal Merchant ...,False,False,False,...,True,False,False,False,False,False,False,False,False,True
2,04-30-22,XL,Shipped,1,INR,329.00,IN Core Free Shipping 2015/04/08 23-48-5-108,True,False,True,...,False,False,False,False,False,False,False,False,False,True
3,04-30-22,L,NaN,0,INR,753.33,NaN,False,True,False,...,True,False,False,False,False,False,False,False,True,False
4,04-30-22,3XL,Shipped,1,INR,574.00,NaN,False,False,True,...,False,False,False,False,False,False,False,True,False,False


In [10]:
# 7. Size - Label Encoding

# Unique Values
unique_sizes = df_amazon["Size"].unique()
print(unique_sizes)

# Counts of values
size_counts = df_amazon["Size"].value_counts()
print(size_counts)

# Map the sizes
size_mapping = {
    "XS": 1,
    "S": 2,
    "M": 3,
    "L": 4,
    "XL": 5,
    "XXL": 6,
    "3XL": 7,
    "4XL": 8,
    "5XL": 9,
    "6XL": 10,
    "Free": 11
}

# Encode it and head
df_amazon["Size"] = df_amazon["Size"].map(size_mapping)
df_amazon.head()

['S' '3XL' 'XL' 'L' 'XXL' 'XS' '6XL' 'M' '4XL' '5XL' 'Free']
Size
M       21928
L       21430
XL      20186
XXL     17529
S       16576
3XL     14371
XS      10849
6XL       722
5XL       532
4XL       411
Free      371
Name: count, dtype: int64


,Date,Size,Courier Status,Qty,currency,Amount,promotion-ids,B2B,Status_Cancelled,Status_Shipped,...,ship-service-level_Standard,Category_Blouse,Category_Bottom,Category_Dupatta,Category_Ethnic Dress,Category_Saree,Category_Set,Category_Top,Category_Western Dress,Category_kurta
0,04-30-22,2,NaN,0,INR,647.62,NaN,False,True,False,...,True,False,False,False,False,False,True,False,False,False
1,04-30-22,7,Shipped,1,INR,406.00,Amazon PLCC Free-Financing Universal Merchant ...,False,False,False,...,True,False,False,False,False,False,False,False,False,True
2,04-30-22,5,Shipped,1,INR,329.00,IN Core Free Shipping 2015/04/08 23-48-5-108,True,False,True,...,False,False,False,False,False,False,False,False,False,True
3,04-30-22,4,NaN,0,INR,753.33,NaN,False,True,False,...,True,False,False,False,False,False,False,False,True,False
4,04-30-22,7,Shipped,1,INR,574.00,NaN,False,False,True,...,False,False,False,False,False,False,False,True,False,False


In [11]:
# 8. Courier Status - Label Encoding

# Unique Values
unique_courier_status = df_amazon["Courier Status"].unique()
print(unique_courier_status)

# Counts of values
courier_status_counts = df_amazon["Courier Status"].value_counts()
print(courier_status_counts)

# Should drop? I will drop it as of now
df_amazon.drop(columns=["Courier Status"], inplace=True)
df_amazon



[nan 'Shipped' 'Cancelled' 'Unshipped']
Courier Status
Shipped      106357
Cancelled      5933
Unshipped      5746
Name: count, dtype: int64


,Date,Size,Qty,currency,Amount,promotion-ids,B2B,Status_Cancelled,Status_Shipped,Status_Shipped - Delivered to Buyer,...,ship-service-level_Standard,Category_Blouse,Category_Bottom,Category_Dupatta,Category_Ethnic Dress,Category_Saree,Category_Set,Category_Top,Category_Western Dress,Category_kurta
0,04-30-22,2,0,INR,647.62,NaN,False,True,False,False,...,True,False,False,False,False,False,True,False,False,False
1,04-30-22,7,1,INR,406.00,Amazon PLCC Free-Financing Universal Merchant ...,False,False,False,True,...,True,False,False,False,False,False,False,False,False,True
2,04-30-22,5,1,INR,329.00,IN Core Free Shipping 2015/04/08 23-48-5-108,True,False,True,False,...,False,False,False,False,False,False,False,False,False,True
3,04-30-22,4,0,INR,753.33,NaN,False,True,False,False,...,True,False,False,False,False,False,False,False,True,False
4,04-30-22,7,1,INR,574.00,NaN,False,False,True,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124900,05-31-22,5,1,INR,517.00,NaN,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
124901,05-31-22,3,1,INR,999.00,IN Core Free Shipping 2015/04/08 23-48-5-108,False,False,True,False,...,False,False,False,False,False,False,True,False,False,False
124902,05-31-22,6,1,INR,690.00,NaN,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
124903,05-31-22,1,1,INR,1199.00,IN Core Free Shipping 2015/04/08 23-48-5-108,False,False,True,False,...,False,False,False,False,False,False,True,False,False,False


In [12]:
# 9. Quantity - Exploring the different quantities bought

# Unique Values
unique_qty = df_amazon["Qty"].unique()
print(unique_qty)

# Counts of values
qty_counts = df_amazon["Qty"].value_counts()
print(qty_counts)


[ 0  1  2 15  3  9 13  5  4  8]
Qty
1     111739
0      12802
2        324
3         26
4          8
5          2
15         1
9          1
13         1
8          1
Name: count, dtype: int64


In [13]:
# 10: Currency Label - Viewing different types, dropping nan

unique_currency = df_amazon["currency"].unique()
print(unique_currency)

# Counts of values
currency_counts = df_amazon["currency"].value_counts()
print(currency_counts)

# Drop rows with nan
df_amazon.dropna(subset=['currency'], inplace=True)

# Drop currency column, all INR
df_amazon.drop(columns=['currency'], inplace=True)
df_amazon.head()



['INR' nan]
currency
INR    117123
Name: count, dtype: int64


,Date,Size,Qty,Amount,promotion-ids,B2B,Status_Cancelled,Status_Shipped,Status_Shipped - Delivered to Buyer,Fulfilment_Amazon,...,ship-service-level_Standard,Category_Blouse,Category_Bottom,Category_Dupatta,Category_Ethnic Dress,Category_Saree,Category_Set,Category_Top,Category_Western Dress,Category_kurta
0,04-30-22,2,0,647.62,NaN,False,True,False,False,False,...,True,False,False,False,False,False,True,False,False,False
1,04-30-22,7,1,406.00,Amazon PLCC Free-Financing Universal Merchant ...,False,False,False,True,False,...,True,False,False,False,False,False,False,False,False,True
2,04-30-22,5,1,329.00,IN Core Free Shipping 2015/04/08 23-48-5-108,True,False,True,False,True,...,False,False,False,False,False,False,False,False,False,True
3,04-30-22,4,0,753.33,NaN,False,True,False,False,False,...,True,False,False,False,False,False,False,False,True,False
4,04-30-22,7,1,574.00,NaN,False,False,True,False,True,...,False,False,False,False,False,False,False,True,False,False


In [14]:
# 11: Promotion IDs - Convert to boolean
df_amazon["promotion-ids"] = df_amazon["promotion-ids"].apply(lambda x: 0 if pd.isna(x) else 1)
df_amazon.head()


,Date,Size,Qty,Amount,promotion-ids,B2B,Status_Cancelled,Status_Shipped,Status_Shipped - Delivered to Buyer,Fulfilment_Amazon,...,ship-service-level_Standard,Category_Blouse,Category_Bottom,Category_Dupatta,Category_Ethnic Dress,Category_Saree,Category_Set,Category_Top,Category_Western Dress,Category_kurta
0,04-30-22,2,0,647.62,0,False,True,False,False,False,...,True,False,False,False,False,False,True,False,False,False
1,04-30-22,7,1,406.00,1,False,False,False,True,False,...,True,False,False,False,False,False,False,False,False,True
2,04-30-22,5,1,329.00,1,True,False,True,False,True,...,False,False,False,False,False,False,False,False,False,True
3,04-30-22,4,0,753.33,0,False,True,False,False,False,...,True,False,False,False,False,False,False,False,True,False
4,04-30-22,7,1,574.00,0,False,False,True,False,True,...,False,False,False,False,False,False,False,True,False,False


In [15]:
# 12: B2B - Convert to boolean
df_amazon["B2B"] = df_amazon["B2B"].astype(int)
df_amazon.head()

,Date,Size,Qty,Amount,promotion-ids,B2B,Status_Cancelled,Status_Shipped,Status_Shipped - Delivered to Buyer,Fulfilment_Amazon,...,ship-service-level_Standard,Category_Blouse,Category_Bottom,Category_Dupatta,Category_Ethnic Dress,Category_Saree,Category_Set,Category_Top,Category_Western Dress,Category_kurta
0,04-30-22,2,0,647.62,0,0,True,False,False,False,...,True,False,False,False,False,False,True,False,False,False
1,04-30-22,7,1,406.00,1,0,False,False,True,False,...,True,False,False,False,False,False,False,False,False,True
2,04-30-22,5,1,329.00,1,1,False,True,False,True,...,False,False,False,False,False,False,False,False,False,True
3,04-30-22,4,0,753.33,0,0,True,False,False,False,...,True,False,False,False,False,False,False,False,True,False
4,04-30-22,7,1,574.00,0,0,False,True,False,True,...,False,False,False,False,False,False,False,True,False,False


In [16]:
# Date: Convert Date to datetime and seperate into year, month, day
df_amazon["Date"] = pd.to_datetime(df_amazon['Date'], format='%m-%d-%y')

# df_amazon['Year'] = df_amazon['Date'].dt.year # Only one year: 2022
df_amazon['Month'] = df_amazon['Date'].dt.month
df_amazon['Day'] = df_amazon['Date'].dt.day

df_amazon.drop(columns=["Date"], inplace=True)

df_amazon

,Size,Qty,Amount,promotion-ids,B2B,Status_Cancelled,Status_Shipped,Status_Shipped - Delivered to Buyer,Fulfilment_Amazon,Fulfilment_Merchant,...,Category_Bottom,Category_Dupatta,Category_Ethnic Dress,Category_Saree,Category_Set,Category_Top,Category_Western Dress,Category_kurta,Month,Day
0,2,0,647.62,0,0,True,False,False,False,True,...,False,False,False,False,True,False,False,False,4,30
1,7,1,406.00,1,0,False,False,True,False,True,...,False,False,False,False,False,False,False,True,4,30
2,5,1,329.00,1,1,False,True,False,True,False,...,False,False,False,False,False,False,False,True,4,30
3,4,0,753.33,0,0,True,False,False,False,True,...,False,False,False,False,False,False,True,False,4,30
4,7,1,574.00,0,0,False,True,False,True,False,...,False,False,False,False,False,True,False,False,4,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124900,5,1,517.00,0,0,False,True,False,True,False,...,False,False,False,False,False,False,False,True,5,31
124901,3,1,999.00,1,0,False,True,False,True,False,...,False,False,False,False,True,False,False,False,5,31
124902,6,1,690.00,0,0,False,True,False,True,False,...,False,False,False,False,False,False,True,False,5,31
124903,1,1,1199.00,1,0,False,True,False,True,False,...,False,False,False,False,True,False,False,False,5,31


In [17]:
# Correlation matrix
df_amazon.corr(numeric_only=True)

,Size,Qty,Amount,promotion-ids,B2B,Status_Cancelled,Status_Shipped,Status_Shipped - Delivered to Buyer,Fulfilment_Amazon,Fulfilment_Merchant,...,Category_Bottom,Category_Dupatta,Category_Ethnic Dress,Category_Saree,Category_Set,Category_Top,Category_Western Dress,Category_kurta,Month,Day
Size,1.000000,0.005153,-0.021174,-0.031839,0.004317,-0.021816,-0.008581,0.024070,-0.020830,0.020830,...,0.002422,0.017858,0.000863,0.127616,-0.092610,-0.000153,-0.020364,0.086648,0.012612,0.001168
Qty,0.005153,1.000000,0.064085,0.278905,0.013801,-0.637055,0.282132,0.117669,0.316172,-0.316172,...,-0.004587,0.000942,0.001297,0.003465,0.000805,0.018151,-0.024604,0.005426,0.013795,-0.012846
Amount,-0.021174,0.064085,1.000000,0.127248,0.015589,-0.005996,0.002762,0.000991,0.008308,-0.008308,...,-0.059684,-0.006184,0.026024,0.019399,0.525655,-0.131388,0.152365,-0.544696,0.052529,0.010466
promotion-ids,-0.031839,0.278905,0.127248,1.000000,-0.000723,-0.427550,-0.122639,0.421715,-0.269829,0.269829,...,0.004346,0.003752,-0.012617,0.007543,0.079984,-0.046891,0.074076,-0.098454,-0.040230,0.009064
B2B,0.004317,0.013801,0.015589,-0.000723,1.000000,-0.010542,-0.003579,0.011008,-0.006356,0.006356,...,0.002054,-0.000426,0.006957,0.002607,0.000026,-0.004268,0.002331,-0.001338,-0.002522,0.006505
Status_Cancelled,-0.021816,-0.637055,-0.005996,-0.427550,-0.010542,1.000000,-0.445776,-0.181515,-0.131647,0.131647,...,0.002094,-0.001610,-0.005649,-0.001689,0.001499,-0.008456,0.004408,0.001647,-0.000612,-0.003082
Status_Shipped,-0.008581,0.282132,0.002762,-0.122639,-0.003579,-0.445776,1.000000,-0.799359,0.894178,-0.894178,...,-0.013808,0.003612,0.008820,0.008816,0.018002,0.038110,-0.100265,0.024509,0.061902,-0.016285
Status_Shipped - Delivered to Buyer,0.024070,0.117669,0.000991,0.421715,0.011008,-0.181515,-0.799359,1.000000,-0.893960,0.893960,...,0.013764,-0.002887,-0.005898,-0.008551,-0.020783,-0.036190,0.107190,-0.028030,-0.067593,0.019959
Fulfilment_Amazon,-0.020830,0.316172,0.008308,-0.269829,-0.006356,-0.131647,0.894178,-0.893960,1.000000,-1.000000,...,-0.016098,0.003230,0.006357,0.007965,0.021247,0.043454,-0.113125,0.027930,0.071140,-0.023914
Fulfilment_Merchant,0.020830,-0.316172,-0.008308,0.269829,0.006356,0.131647,-0.894178,0.893960,-1.000000,1.000000,...,0.016098,-0.003230,-0.006357,-0.007965,-0.021247,-0.043454,0.113125,-0.027930,-0.071140,0.023914


In [31]:
# Export to csv
df_amazon.to_csv("dataset/eda_amazon_sales_report.csv")